<a href="https://colab.research.google.com/github/simecek/dspracticum2024/blob/main/lesson02/FashionMNIST_SimpleCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let's build a simple neural network to classify images from the FashionMNIST dataset.

**1. Import Libraries**

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

*Checking for GPU Availability*

This code checks if a CUDA-enabled GPU is available and sets the `device` accordingly. If no GPU is available, it defaults to the CPU.

In [9]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [ ]:
config = {
    'input_channels': 1,
    'output_channels1': 32,
    'output_channels2': 64,
    'kernel_size': 3,
    'flatten_dim': 24*24*64,
    'fc1_output_dim': 128,
    'dropout_prob': 0.3,
    'fc2_output_dim': 10,
    'learning_rate': 0.01,
    'batch_size': 16,
    'epochs': 25
}

**2. Data Preparation**

In [3]:
transform = transforms.ToTensor()

# Download and load the training data
train_set = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=config['batch_size'], shuffle=True)

# Download and load the test data
test_set = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=config['batch_size'], shuffle=False)

**3. Neural Network Model**

In [4]:
class SimpleCNN(nn.Module):
    def __init__(self, config):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=config['input_channels'], out_channels=config['output_channels1'], kernel_size=config['kernel_size'])
        self.conv2 = nn.Conv2d(in_channels=config['output_channels1'], out_channels=config['output_channels2'], kernel_size=config['kernel_size'])
        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(in_features=config['flatten_dim'], out_features=config['fc1_output_dim'])
        self.drop = nn.Dropout(config['dropout_prob'])
        self.fc2 = nn.Linear(in_features=config['fc1_output_dim'], out_features=config['fc2_output_dim'])

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.flat(x)
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = self.fc2(x)
        return x

model = SimpleCNN(config).to(device)
model

SimpleCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (flat): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=36864, out_features=128, bias=True)
  (drop): Dropout(p=0.25, inplace=False)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

**4. Loss & Optimizer**

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=config['learning_rate'])

**5. Training loop**

In [6]:
epochs = config['epochs']

for epoch in range(epochs):  # Train for 5 epochs
    running_loss = 0.0
    for images, labels in train_loader:
        # Move images and labels to the device
        images = images.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}')

Epoch [1/10], Loss: 0.9873
Epoch [2/10], Loss: 0.6152
Epoch [3/10], Loss: 0.5532
Epoch [4/10], Loss: 0.5166
Epoch [5/10], Loss: 0.4901
Epoch [6/10], Loss: 0.4772
Epoch [7/10], Loss: 0.4601
Epoch [8/10], Loss: 0.4442
Epoch [9/10], Loss: 0.4340
Epoch [10/10], Loss: 0.4256


**6. Evaluation on the test set**

In [7]:
correct = 0
total = 0
with torch.no_grad():  # Disable gradient calculation for evaluation
    for images, labels in test_loader:
        # Move images and labels to the device
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total:.2f}%')

Accuracy: 82.89%
